In [1]:
import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm as tqdm
from helpers.rotation import RotateFromTo

In [2]:
data = pv.read("../../real_data/GrotteDeLaCascadeDeMotiers/gallery/3e_Galerie_cropped.stl").extract_surface().triangulate()

In [3]:
GLOBAL_SHIFT = [-537800, -195100, 0]

In [4]:
plane_origins = np.loadtxt("../../real_data/GrotteDeLaCascadeDeMotiers/planes_as_vectors/3rdGallery_Planes_Origin.txt") + GLOBAL_SHIFT
plane_vectors =  np.loadtxt("../../real_data/GrotteDeLaCascadeDeMotiers/planes_as_vectors/3rdGallery_Planes.txt")

In [5]:
# define orthogonal unit vectors in horizontal plane 
d0 = np.array([1,0,0])
d1 = np.array([0,1,0])

# project the section-normal vectors onto horizontal plane. 
horiz_dip_dir = np.asarray((plane_vectors.dot(d0), plane_vectors.dot(d1), np.zeros(len(plane_vectors)))).T

In [ ]:
for n, (orig, vec, vec_horiz) in tqdm(enumerate(zip(plane_origins, plane_vectors, horiz_dip_dir)), total = len(plane_origins)):
    plane = pv.Plane(orig, vec, 20, 20, 100, 100).extract_surface().triangulate()
    inter, _, _ = data.intersection(plane, split_first = False, split_second = False)

    R1 =  RotateFromTo(vec, vec_horiz)
    pp = inter.points - np.mean(plane.points, axis=0)
    pp_vertical = R1 @ pp.T
    R2 = RotateFromTo(vec_horiz, d1)
    pp_rotated = R2 @ pp_vertical

    np.savetxt(f"../../real_data/GrotteDeLaCascadeDeMotiers/sections_as_polylines/unordered_section_{n}.txt", pp_rotated.T)
        
    #pp_reordered = order_points(pp_rotated.T.tolist(), 0)


 39%|███████████████████████████████▋                                                 | 66/169 [07:14<11:16,  6.57s/it]